# Support vector machine

En el presente *notebook* se va a resolver un problema de clasificación con SVM. El dataset que se va a emplear es "*healthcare-dataset-stroke-data*".

## Dataset para la predicción de accidentes cerebrovasculares

El dataset *Stroke Prediction* está compuesto de 11 variables clinicas para prevenir los derrames cerebrales. Dichas variables son:

1.   *id*: identificador
2.   *gender*: "Male" (masculino), "Female" (femenino) u "Other" (otro)
3.   *age*: Edad del paciente
4.   *hypertension*: 0 si el paciente no tiene hipertensión, 1 en caso contrario
5.   *heart_disease*: 0 si el paciente no padece enfermedades del corazón, 1 en caso contrario
6.   *ever_married*: "No" o "Yes"
7.   *work_type*: Tipo de trabajo. Los valores que puede tomar son: "children", "Govt_jov", "Never_worked", "Private" o "Self-employed"
8.   *Residence_type*: Tipo de residencia, puede ser: "Rural" o "Urban"
9.   *avg_glucose_level*: Promedio del nivel de glucosa en la sangre
10.   *bmi*: índice de masa corporal
11.   *smoking_status*: Indica si el paciente a fumado o no, los valores que puede tomar son: "formerly smoked" (antes fumaba), "never smoked" (nunca ha fumado), "smokes" (fuma) or "Unknown" (desconocido)
12.   **stroke**: 1 si el paciente ha sufrido un derrame cerebral o 0 si no



NOTA: "Unknown" La información del paciente no se encuentra disponible.

FUENTE: https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset?resource=download#




In [1]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df_healthcare = pd.read_csv('./drive/MyDrive/ML/healthcare-dataset-stroke-data.csv')

## Tratamiento de nulos

La columna que tienen datos nulos es *'bmi'*, así que los datos faltantes se van a reemplazar con la media considerando la edad (*age*) y el genero (*gender*).

In [4]:
df_mean_bmi = df_healthcare.groupby(['age', 'gender'])['bmi'].mean()
df_mean_bmi.head()

age   gender
0.08  Female    14.100000
      Male      16.900000
0.16  Male      14.766667
0.24  Male      17.400000
0.32  Female    17.266667
Name: bmi, dtype: float64

In [5]:
# La siguente función sustituye los null por la media calculada en el dataframe anterior.

def sustituir_nan_bmi(df, df_mean_bmi):
    if np.isnan(df['bmi']):
        return df_mean_bmi[df['age']][df['gender']]
    else:
        return df['bmi']

In [6]:
df_healthcare['bmi'] = df_healthcare.apply(sustituir_nan_bmi, axis=1, args=(df_mean_bmi, ))
df_healthcare.isna().sum()

,0
id,0
gender,0
age,0
hypertension,0
heart_disease,0
ever_married,0
work_type,0
Residence_type,0
avg_glucose_level,0
bmi,1


In [7]:
# Se filtra que registro aún es null
df_healthcare[df_healthcare['bmi'].isnull()]

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
2030,38920,Male,0.48,0,0,No,children,Urban,73.02,NaN,Unknown,0


In [8]:
# Dado que la media del último no corresponde con el genero "Male", la asignación se hace de manera manual.
df_healthcare.loc[2030, 'bmi'] = df_mean_bmi[0.48]['Female']

Eliminar columna id, ya que no aporta información. Además de la fila que
tiene en *'gender'* el valor de "***Other***"

In [9]:
df_healthcare = df_healthcare.drop(columns=['id'])
idx = df_healthcare.index[df_healthcare["gender"]=="Other"].tolist()
df_healthcare = df_healthcare.drop(idx)

In [10]:
df_healthcare.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5109 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             5109 non-null   object 
 1   age                5109 non-null   float64
 2   hypertension       5109 non-null   int64  
 3   heart_disease      5109 non-null   int64  
 4   ever_married       5109 non-null   object 
 5   work_type          5109 non-null   object 
 6   Residence_type     5109 non-null   object 
 7   avg_glucose_level  5109 non-null   float64
 8   bmi                5109 non-null   float64
 9   smoking_status     5109 non-null   object 
 10  stroke             5109 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 479.0+ KB


## Desbalanceo de datos

In [11]:
df_healthcare['stroke'].value_counts()

,count
stroke,
0,4860
1,249


## Preprocesamiento de dataset

### Variables categóricas

In [12]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df_categorical = df_healthcare.select_dtypes(include=['object']).apply(label_encoder.fit_transform)
df_healthcare[df_categorical.columns] = df_categorical

In [13]:
df_categorical.head()

,gender,ever_married,work_type,Residence_type,smoking_status
0,1,1,2,1,1
1,0,1,3,0,2
2,1,1,2,0,2
3,0,1,2,1,3
4,0,1,3,0,2


In [14]:
'''
data = pd.get_dummies(df_healthcare['gender'], drop_first=True)
df_healthcare = pd.concat([df_healthcare, data], axis=1)

data = pd.get_dummies(df_healthcare['ever_married'], drop_first=True)
df_healthcare = pd.concat([df_healthcare, data], axis=1)

data = pd.get_dummies(df_healthcare['work_type'], drop_first=True)
df_healthcare = pd.concat([df_healthcare, data], axis=1)

data = pd.get_dummies(df_healthcare['Residence_type'], drop_first=True)
df_healthcare = pd.concat([df_healthcare, data], axis=1)

data = pd.get_dummies(df_healthcare['smoking_status'], drop_first=True)
df_healthcare = pd.concat([df_healthcare, data], axis=1)
'''

"\ndata = pd.get_dummies(df_healthcare['gender'], drop_first=True)\ndf_healthcare = pd.concat([df_healthcare, data], axis=1)\n\ndata = pd.get_dummies(df_healthcare['ever_married'], drop_first=True)\ndf_healthcare = pd.concat([df_healthcare, data], axis=1)\n\ndata = pd.get_dummies(df_healthcare['work_type'], drop_first=True)\ndf_healthcare = pd.concat([df_healthcare, data], axis=1)\n\ndata = pd.get_dummies(df_healthcare['Residence_type'], drop_first=True)\ndf_healthcare = pd.concat([df_healthcare, data], axis=1)\n\ndata = pd.get_dummies(df_healthcare['smoking_status'], drop_first=True)\ndf_healthcare = pd.concat([df_healthcare, data], axis=1)\n"

In [15]:
#df_healthcare.drop(['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status'], axis=1, inplace=True)

## Dividir el dataset

In [16]:
from sklearn.model_selection import train_test_split

X = df_healthcare.drop('stroke', axis=1)
y = df_healthcare['stroke']

# Se divide el dataset en 2 partes:
# 1. Train (entrenamiento)
# 2. Test (prueba)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y, random_state=42)

## Normalización de datos

In [ ]:
from sklearn.preprocessing import Normalizer # StandardScaler # MinMaxScaler

norm = Normalizer() #StandardScaler() #MinMaxScaler()
norm.fit(X_train)
X_train = norm.transform(X_train)
X_test = norm.transform(X_test)

In [18]:
from sklearn import svm

# Modelos SVM en scikit-learn

Scikit-learn ofrece tres implementaciones de **Máquinas de Vectores de Soporte (SVM)** para problemas de clasificación binaria o multiclase:


<ol>
  <li>SVC (Support Vector Classification)</li>
  <ul>
    <li> Parámetro clave: C (regularización)</li>
    <li> Más popular y rápido que NuSVC</li>
    <li> Soporta múltiples kernels (linear, rbf, poly, sigmoid)</li>
  </ul>
  <li>NuSVC (Nu-Support Vector Classification)</li>
  <ul>
    <li> Parámetro clave: nu (control de vectores de soporte)</li>
    <li> Requiere validación matemática de factibilidad</li>
    <li> Menos utilizado que SVC</li>
  </ul>
  <li> LinearSVC (Linear Support Vector Classification)</li>
  <ul>
    <li> Solo kernel lineal (optimizado para este caso)</li>
    <li> Más eficiente en memoria y para datasets grandes</li>
  </ul>
</ol>



## Método SVC

### Parámetros del modelo SVC

* C - (Flotante) Parámetro de regularización, por default es 1.0
* kernel - Tipo de kernel a usar, puede ser: "linear", "poly", "rbf" y "sigmoid", por default es "rbf"
* degree - Grado del polinomio (kernel polinomial), por default 3
* gamma - Coeficiente para los kernels "poly", "rbf" y "sigmoid". Los valores pueden ser: "scale", "auto" o un número flotante no negativo.
* coef0 - (Flotante) Termino independiente en la función del kernel. Aplica a "poly" y "sigmoid", por defaul es 0.0
* tol - (Flotante) Tolerancia para el criterio de parada, por default (1e-3)
* cache_size - (Flotante) Especifica el tamaño del kernel en cache (MB), por default 200
* verbose - Habilita la salida
* max_iter - (Entero) Máximo número de iteraciones
* random_state - (Entero) Semilla

### Parámetro C (Regularización)

<ul>
  <li><b>Definición</b>: Controla el trade-off entre margen y errores de clasificación.</li>
  <ul>
    <li> C <b>alto</b>: Margen estrecho → Prioriza clasificar correctamente todos los puntos (riesgo de sobreajuste).
    </li>
    <li> C <b>bajo</b>: Margen amplio → Permite más errores (mejor generalización).</li>
  </ul>
  <li> <b>Nota</b>: No tiene interpretación probabilística.</li>
</ul>

### Modelo 1

In [19]:
# El parámetro class_weight se utiliza en algoritmos de clasificación cuando el dataset está desbalanceado.
# Este parámetro asigna un peso a cada clase, calculado en función de su proporción en el conjunto de datos.
# Su propósito es indicarle al modelo que preste mayor atención a las clases menos representadas,
# compensando así el desbalance durante el entrenamiento.

from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight(
    class_weight = "balanced",
    classes = np.unique(y_train),
    y = y_train
)
class_weights = dict(zip(set(y_train), class_weights))
print(class_weights)

{0: np.float64(0.5255915637860082), 1: np.float64(10.268844221105528)}


In [20]:
# El método empleado para instanciar una SVM de clasificación es SVC()
svm_healthy = svm.SVC(gamma='scale',  class_weight=class_weights, verbose=True)
#svm_healthy = svm.SVC(gamma='scale', verbose=True)

In [21]:
# Para entrenar el modelo, se usa el método fit() que requiere 2 conjuntos de datos X_train y y_train

svm_healthy.fit(X_train, y_train)

[LibSVM]

SVC(class_weight={0: np.float64(0.5255915637860082),
                  1: np.float64(10.268844221105528)},
    verbose=True)

In [22]:
# Para realizar inferencias con datos nuevos, se emplea el método predict()

predict_svm = svm_healthy.predict(X_test)

### Modelo 2

In [23]:
svc_healthy_2 = svm.SVC(C=10, cache_size=100, class_weight=None, coef0=0.5, degree=5, gamma='scale', kernel='poly',
    max_iter=200, random_state=103, tol=0.01, verbose=True)

In [24]:
svc_healthy_2.fit(X_train, y_train)

[LibSVM]

/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVC(C=10, cache_size=100, coef0=0.5, degree=5, kernel='poly', max_iter=200,
    random_state=103, tol=0.01, verbose=True)

In [25]:
# Para realizar inferencias con datos nuevos, se emplea el método predict()

predict_svm_2 = svc_healthy_2.predict(X_test)

## GridSearchCV

Permite probar de manera secuencial una combinación de hiperparámetros

In [26]:
from sklearn.model_selection import GridSearchCV

# Parametros y los valores que van a tomar
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'kernel': ['rbf', 'sigmoid'],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001]
              }

grid = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 0)

# fitting the model for grid search
# Para que no se impriman los resultados, estos se asignan a _
_ = grid.fit(X_train, y_train)

# Se asignan los resultados a un dataframe
resultados = pd.DataFrame(grid.cv_results_)
resultados.filter(regex = '(param.*|mean_t|std_t)')\
    .drop(columns = 'params')\
    .sort_values('mean_test_score', ascending = False) \
    .head(5)

,param_C,param_gamma,param_kernel,mean_test_score,std_test_score
0,0.1,1.00,rbf,0.951309,0.000478
1,0.1,1.00,sigmoid,0.951309,0.000478
2,0.1,0.10,rbf,0.951309,0.000478
3,0.1,0.10,sigmoid,0.951309,0.000478
4,0.1,0.01,rbf,0.951309,0.000478


In [27]:
# print best parameter after tuning
print(f'Mejores hiperparámetros {grid.best_params_}\nAccuracy: {grid.best_score_} {grid.scoring}')

# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

Mejores hiperparámetros {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
Accuracy: 0.9513091308472467 None
SVC(C=0.1, gamma=1)


### Modelo 3

In [28]:
modelo_best_grid = grid.best_estimator_
print(modelo_best_grid.get_params())

{'C': 0.1, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 1, 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}


In [29]:
predict_best = modelo_best_grid.predict(X_test)


## Reportes de métricas

In [30]:
from sklearn.metrics import classification_report
class_label = ['False', 'True']

In [31]:
print("Modelo 1")
print(classification_report(y_test, predict_svm, target_names=class_label))

Modelo 1
              precision    recall  f1-score   support

       False       0.98      0.61      0.75       972
        True       0.09      0.78      0.17        50

    accuracy                           0.62      1022
   macro avg       0.54      0.69      0.46      1022
weighted avg       0.94      0.62      0.72      1022



In [32]:
print("Modelo 2")
print(classification_report(y_test, predict_svm_2, target_names=class_label))

Modelo 2
              precision    recall  f1-score   support

       False       0.93      0.35      0.50       972
        True       0.04      0.48      0.07        50

    accuracy                           0.35      1022
   macro avg       0.48      0.41      0.29      1022
weighted avg       0.88      0.35      0.48      1022



In [33]:
print("Modelo 3")
print(classification_report(y_test, predict_best, target_names=class_label))

Modelo 3
              precision    recall  f1-score   support

       False       0.95      1.00      0.97       972
        True       0.00      0.00      0.00        50

    accuracy                           0.95      1022
   macro avg       0.48      0.50      0.49      1022
weighted avg       0.90      0.95      0.93      1022



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
